# 상위권 예측 결과 분석

### prediction.json 병합

In [33]:
import json
import os
from collections import defaultdict


def merge_json_files(directory_path, output_file):
    # 결과를 저장할 딕셔너리 생성
    merged_data = defaultdict(list)

    # 디렉토리 내의 모든 JSON 파일 처리
    for filename in os.listdir(directory_path):
        if filename.endswith(".json"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, "r", encoding="utf-8") as file:
                data = json.load(file)
                # 각 키-값 쌍을 merged_data에 추가
                for key, value in data.items():
                    merged_data[key].append(value)

    # 결과를 새 JSON 파일로 저장
    with open(output_file, "w", encoding="utf-8") as outfile:
        json.dump(dict(merged_data), outfile, ensure_ascii=False, indent=4)

    print(f"병합된 데이터가 {output_file}에 저장되었습니다.")

In [34]:
# json 병합
directory_path = "./data/hard_ensemble"  # JSON 파일이 있는 디렉토리 경로
output_file = "./data/merged_output.json"  # 결과를 저장할 파일 이름
merge_json_files(directory_path, output_file)

병합된 데이터가 ./data/merged_output.json에 저장되었습니다.


### 난이도별로 json 분리

In [35]:
from collections import Counter, defaultdict


def classify_difficulty(data):
    difficulty_levels = defaultdict(list)

    for key, values in data.items():
        value_counts = Counter(values)
        max_count = max(value_counts.values())

        if max_count <= 2:
            difficulty_levels["어려움"].append({key: values})
        elif 3 <= max_count <= 5:
            difficulty_levels["중간"].append({key: values})
        elif 6 <= max_count <= 8:
            difficulty_levels["쉬움"].append({key: values})

    return difficulty_levels


def save_difficulty_levels(difficulty_levels):
    for difficulty, items in difficulty_levels.items():
        filename = f"{difficulty}.json"
        with open(filename, "w", encoding="utf-8") as outfile:
            json.dump(items, outfile, ensure_ascii=False, indent=4)

In [ ]:
# json 파일 불러오기
file_path = "./data/merged_output.json"
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)
# 난이도 분류
classified_data = classify_difficulty(data)
# 분류된 데이터 저장
save_difficulty_levels(classified_data)
print("데이터가 난이도별로 분류되어 저장되었습니다.")

### Question & Prediction 분석

In [36]:
import pandas as pd

pd.options.display.max_colwidth = 200
pd.set_option("display.max_rows", 30)

In [45]:
import pandas as pd
import json
from collections import Counter

# CSV 파일 로드
df = pd.read_csv("data/test_validation_dataset.csv")

# JSON 파일 로드
with open("data/merged_output.json", "r", encoding="utf-8") as f:
    json_data = json.load(f)


# 동일 예측 개수 반환
def determine_difficulty(values):
    value_counts = Counter(values)
    max_count = max(value_counts.values())
    return max_count

# predictions와 difficulty 컬럼 추가
df["predictions"] = df["id"].map(json_data)
df["difficulty"] = df["predictions"].apply(determine_difficulty)

# 컬럼 위치 변경
df = df[["id", "question", "predictions", "difficulty"]]

df.head()

,id,question,predictions,difficulty
0,mrc-1-000653,유령'은 어느 행성에서 지구로 왔는가?,"[우스, 사만 행성, 사만, 베텔게우스(Betelgeuse) 근처의 작은 행성, 지구, 사만, 사만 행성]",2
1,mrc-1-001113,용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?,"[냉전, 냉전, 냉전, 냉전, 냉전, 냉전, 냉전 종식]",6
2,mrc-0-002191,돌푸스에게 불특정 기간동안 하원이 잠시 쉬는 것을 건의 받았던 인물은?,"[빌헬름 미클라스, 빌헬름 미클라스, 대통령인 빌헬름 미클라스, 대통령인 빌헬름 미클라스, 대통령인 빌헬름 미클라스, 대통령인 빌헬름 미클라스, 대통령인 빌헬름 미클라스]",5
3,mrc-0-003951,"마오리언어와 영어, 뉴질랜드 수화를 공식 언어로 사용하는 나라는?","[뉴질랜드, 뉴질랜드, 뉴질랜드, 뉴질랜드, 뉴질랜드, 뉴질랜드, 뉴질랜드]",7
4,mrc-1-001272,디엔비엔푸 전투에서 보응우옌잡이 상대한 국가는?,"[프랑스, 프랑스, 프랑스군, 프랑스, 프랑스, 프랑스, 프랑스]",6


In [46]:
df["difficulty"].value_counts()

difficulty
7    241
6     90
5     77
3     77
4     76
2     30
1      9
Name: count, dtype: int64

In [47]:
# 혼자 다른 값을 가진 인덱스의 빈도를 계산하는 함수
def count_unique_indices(predictions_list):
    n = len(predictions_list)
    counts = [0] * n

    for i in range(n):
        if predictions_list.count(predictions_list[i]) == 1:
            counts[i] += 1

    return counts


# 전체 데이터프레임에 대해 인덱스별 카운트 계산
all_counts = df["predictions"].apply(count_unique_indices)

# 인덱스별 총 카운트 계산
total_counts = [sum(counts) for counts in zip(*all_counts)]

print("각 인덱스별 혼자 다른 값의 총 개수:")
for idx, count in enumerate(total_counts):
    print(f"Index {idx}: {count}")


각 인덱스별 혼자 다른 값의 총 개수:
Index 0: 105
Index 1: 76
Index 2: 76
Index 3: 80
Index 4: 103
Index 5: 63
Index 6: 77


In [39]:
df[df["difficulty"]==1]

,id,question,predictions,difficulty
7,mrc-1-000163,아시노 호가 만들어진 원인은 무엇인가?,"[무명, 호조 마사코, 인격적 존재, 시공을 초월한 영원한 존재, 지질, 선종 조사 초상에 연고를 두고 있기 때문, 유루(有漏)]",1
150,mrc-1-000116,튀폰의 아버지는 누구인가?,"[대지의 여신 가이아, 카를 마르텔, 가이아, 타르타로스, 제우스, 제우스가 크로노스를 물리치고 신들의 지배자 자리에 오르자 이에 분노하여 크로노스의 원수를 갚기 위해 그녀의 또 다른 배우자인 타르타로스, 프랑스 귀족 생시몽 공작 클로드]",1
256,mrc-0-002231,정치적 코미트를 담고있는 작품은?,"[바이스, <고골에게 보내는 편지>, 벨린스키의 <고골에게 보내는 편지>, <마라의 박해와 암살>, ≪악령≫, 격언시, 싱글 Shipbuilding]",1
300,mrc-0-004196,루이가 사망하게 될 원인은 무엇인가?,"[식중독, 헬리콥터 추락사고, 중풍, 독살, 요로감염증, 중풍과 등창, 노환 등의 합병증, 합병증]",1
460,mrc-1-000870,아즈마의 초기 명칭은?,"[이즈마 하치로로, '주즈 알-나시샤바', ≪바람 속에서 벌거벗고(Naked in the Wind)≫, 마하프라즈냐파라미타샤스트라(Mahaprajnaparamita­sastra), Mustang (야생마), 마하프라즈냐파라미타샤스트라, 코우테츠]",1
472,mrc-1-000278,청구취지는 밝히는 방법은?,"['청구의 원인', '청구의 원인' 기재, 정보공개, 충분한 시간, 소송, 구술, 사전 구속 영장]",1
528,mrc-0-000030,수사들이 전쟁의 대가로 지은 건물은?,"[충무사(忠武祠), 평창향교, 크로메르지시궁, 아타우알파, 임진왜란, 영천 숭렬당, 충무사]",1
538,mrc-0-004085,맥국이라는 명칭을 알린 사람은?,"[실학자들, 표트르 1세, 제임스 맥헨리, 지바 쿤다(Ziva Kunda), R. J. 셰퍼, 프랭크 맥린, 실학자]",1
583,mrc-0-001659,인간에게 불타는 느낌의 아픔을 주는 것은?,"[유산, 위대한 유토피아, 세계, 이중예정론, 백루검, 바위, 솔레놉신이]",1


In [40]:
df[df["difficulty"] == 2]

,id,question,predictions,difficulty
0,mrc-1-000653,유령'은 어느 행성에서 지구로 왔는가?,"[우스, 사만 행성, 사만, 베텔게우스(Betelgeuse) 근처의 작은 행성, 지구, 사만, 사만 행성]",2
16,mrc-0-000266,통대를 뽑을 수 있었던 주체는 누구인가?,"[국민, 데르브포르갈, 학교, 충분한 시간, 앙리 브리송 행정부, 하느님, 국민]",2
57,mrc-0-003763,불상에 대한 정보가 적혀있는 것은?,"[묘법연화경, 광배(光背)의 뒷면, 중수 발원문, 광배(光背)의 뒷면, 조성발원기에, 중수 발원문, 복장유물]",2
74,mrc-0-004430,어머니를 죽음에 이르게 한 것은 아들이 무엇처럼 행동했기 때문인가?,"[길거리 불한당, 자신의 감정을 드러내보인 적이 없었기 때문, 교육 평론가, 제우스, 정신이상자, 골수암, 길거리 불한당]",2
85,mrc-1-000813,태풍 초이완의 경보는 어떤 순간부터 풀렸는가?,"[9월 16일 오후, 5월 18일, 9월 14일, 9월 16일 오후, 9월 13일, 9월 13일, 10월 10일 21시경]",2
134,mrc-0-001034,공소권이 사라질 경우 무엇에서 벗어나게 되는가?,"['죄를 범한 자', 폭행, 피고인, 검찰의 공소 유지에 의한 명백한 한계, 피고인, 흉기를 사용하며 협박한 부분이 무혐의, '죄를 범한 자']",2
155,mrc-0-001884,레빌 장군은 무엇이 없었기 때문에 실망했나?,"[전차포와 부무장을 '인간형'의 상반신, '인간형'의 상반신, 복잡한 2족 보행기구, 복잡한 2족 보행기구, 유효한 요격 방법, 하반신, 하반신]",2
166,mrc-0-003205,석조지장보살상이 조각되지 않았을 것으로 추정되는 시기는?,"[조선후기에, 17세기, 17세기, 조선후기에, 18세기 전반, 조선 초기, 1646년]",2
214,mrc-1-000256,"본인의 일에 진중하게 임했고 많은 것을 연습했지만, 자신의 옷차림에 불만을 제기했던 인물은?","[매소드, 대마녀, 를르네 지카르트, 이재현, 쿠니스, 마튀랭이, 쿠니스]",2
263,mrc-1-001797,이경언이 사망한 장소는?,"[감옥, 청주, 청주, 서구 감천동 뒷산, 미얀마 아웅산 묘역, 서구 감천동 뒷산, 감옥]",2


## 가설
1. 여러 모델이 똑같이 답변한 예측은 정답일 확률이 높고, 답이 갈리면 오답일 확률이 높을 것이다.
2. 여러 모델이 똑같이 답변한 예측은 난이도가 쉬운 편일 것이고, 답이 갈리면 난이도가 높을 것이다.
3. 모든 모델이 다 틀리는 경우는 거의 없을 것이다. 적어도 하나의 모델은 정답을 예측했을 것이다.
4. 난이도를 분류할 수 있고, 쉬운 문제만 잘 푸는 모델과 어려운 문제만 잘 푸는 모델이 존재한다면 계층적으로 앙상블을 적용할 수 있을 것이다.

## 궁금증
1. 쉬운 문제만 잘 푸는 모델이 있지 않을까?
2. 쉬운 문제를 틀리면서 어려운 문제 잘 맞추는 모델이 존재할 수 있을까? 혹은 어려운 문제를 잘 풀면 쉬운 문제도 잘 풀지 않을까?

# 최상위 두 예측 비교

In [53]:
import json


def merge_and_split_json(file1_path, file2_path, same_output_path, diff_output_path):
    # 두 JSON 파일 읽기
    with open(file1_path, "r", encoding="utf-8") as f1, open(
        file2_path, "r", encoding="utf-8"
    ) as f2:
        data1 = json.load(f1)
        data2 = json.load(f2)

    same_values = {}
    diff_values = {}

    # 모든 키에 대해 반복
    all_keys = set(data1.keys()) | set(data2.keys())
    for key in all_keys:
        value1 = data1.get(key)
        value2 = data2.get(key)

        if value1 == value2:
            # 값이 같은 경우
            same_values[key] = value1
        else:
            # 값이 다른 경우
            diff_values[key] = [value1, value2]

    # 결과를 JSON 파일로 저장
    with open(same_output_path, "w", encoding="utf-8") as f_same:
        json.dump(same_values, f_same, ensure_ascii=False, indent=2)

    with open(diff_output_path, "w", encoding="utf-8") as f_diff:
        json.dump(diff_values, f_diff, ensure_ascii=False, indent=2)

    print(f"같은 값을 가진 항목들은 {same_output_path}에 저장되었습니다.")
    print(f"다른 값을 가진 항목들은 {diff_output_path}에 저장되었습니다.")


# 사용 예시
file1_path = "data/output1.json"
file2_path = "data/output2.json"
same_output_path = "data/same_values.json"
diff_output_path = "data/diff_values.json"

merge_and_split_json(file1_path, file2_path, same_output_path, diff_output_path)

같은 값을 가진 항목들은 data/same_values.json에 저장되었습니다.
다른 값을 가진 항목들은 data/diff_values.json에 저장되었습니다.


In [68]:
with open(diff_output_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print(len(data))

df = pd.read_csv("data/test_validation_dataset.csv")
df["predictions"] = df["id"].map(data)
df = df.dropna()
df = df[["id", "question", "predictions"]]

32


In [70]:
pd.set_option("display.max_rows", 32)
df

,id,question,predictions
27,mrc-0-001206,펌프가 발매된 해는 언제인가?,"[2004년, 2007년]"
30,mrc-0-003133,아델리 펭귄의 목숨에 지장을 주는 사람의 업종은 무엇인가요?,"[책, 불법 어업]"
93,mrc-0-003670,최고령 초등학생으로 기네스북에 올라있는 사람은?,"[퍼스트 그레이더, ‘마루게’]"
129,mrc-0-001241,협동 화폐 거래소의 인정은 어느 부문에서 나타나는 변화인가?,"[환율 부문, 상업 분야]"
150,mrc-1-000116,튀폰의 아버지는 누구인가?,"[대지의 여신 가이아, 타르타로스]"
162,mrc-0-004626,장정기가 들어있는 우산이끼의 가지는?,"[웅기탁(수그루), 웅기탁]"
172,mrc-0-003614,호르몬의 주요 생성장소가 위치한 곳은?,"[복벽 내, 생식샘]"
186,mrc-0-004460,주인규는 무엇 때문에 스스로 생을 마감하게 되는가?,"[고문, 말기 폐암]"
259,mrc-0-001220,양산이씨 종가가 보유한 왕지 중 양산이씨의 시조와 관련한 왕지는 몇 점이 있는가?,"[4점, 1점]"
280,mrc-0-004738,주권재민의 원칙은 어디에 명시되어 있는가?,"[헌법, 일본국헌법]"


### 질문과 답변의 유사성, 그리고 답변간의 유사성을 통해 Retriever가 틀린 Context를 가져왔는지 여부와 두 모델이 동일한 Context를 참고했는지 등을 추정할 수 있다.
- 예를 들어 `도스토옙스키를 연구할 때 큰 도움이 됐던 책은?` -> `[『훈민정음』, 《작가의 일기》]`는 이는 서로 다른 문서에서 정답을 가져왔을 것이다.
- `주권재민의 원칙은 어디에 명시되어 있는가?` -> `[헌법, 일본국헌법]`는 같은 문서지만 Reader가 서로 다른 정답을 추정한 것이다.